In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")


In [10]:
data = pd.read_csv('/Users/mynimbus/Library/Mobile Documents/com~apple~CloudDocs/GitHub Uploads/StoreSight/0_data/Global_SuperStore_16.csv')
data.head()

,Unnamed: 0,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Postal Code,...,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Returned
0,0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,...,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High,NaN
1,1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,...,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical,NaN
2,2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,...,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium,NaN
3,3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,...,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium,NaN
4,4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,...,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical,NaN


### Data Cleaning & Feature Engineering 

In [11]:
# 1. Drop unnecessary columns
data.drop(columns=['Unnamed: 0', 'Row ID'], inplace=True)

# 2. Convert Order Date and Ship Date to datetime
data['Order Date'] = pd.to_datetime(data['Order Date'], errors='coerce')
data['Ship Date'] = pd.to_datetime(data['Ship Date'], errors='coerce')

# 3. Feature: Shipping Delay (in days)
data['Shipping Delay (Days)'] = (data['Ship Date'] - data['Order Date']).dt.days

# 4. Feature: Profit Margin (as a decimal)
data['Profit Margin'] = data['Profit'] / data['Sales']
data['Profit Margin'] = data['Profit Margin'].replace([np.inf, -np.inf], np.nan)

# 5. Feature: Total Order Value = Sales + Shipping Cost
data['Total Order Value'] = data['Sales'] + data['Shipping Cost']

# 6. Clean column names (optional best practice)
data.columns = data.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

# 7. Show summary
data[['Order_ID', 'Order_Date', 'Ship_Date', 'Shipping_Delay_Days', 'Profit_Margin', 'Total_Order_Value']].head()

,Order_ID,Order_Date,Ship_Date,Shipping_Delay_Days,Profit_Margin,Total_Order_Value
0,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,2,0.280000,262.750
1,IN-2014-JR162107-41675,2014-02-05,2014-02-07,2,-0.077847,4633.025
2,IN-2014-CR127307-41929,2014-10-17,2014-10-18,1,0.177766,6090.661
3,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,2,-0.033376,3802.670
4,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,1,0.109963,3736.000


- Cleaned datetime fields.
- Two new key features:
- Profit_Margin
- Total_Order_Value
- Shipping_Delay_Days
- Removed unnecessary index-related columns.


In [12]:
# 1. Check total missing values
missing = data.isnull().sum()
missing_percent = (missing / len(data)) * 100
missing_df = pd.DataFrame({'Missing_Values': missing, 'Percentage': missing_percent})
missing_df = missing_df[missing_df['Missing_Values'] > 0].sort_values(by='Percentage', ascending=False)
missing_df

,Missing_Values,Percentage
Returned,49088,95.706765
Postal_Code,41296,80.514720


In [13]:
# 2. Handling 'Postal_Code': Drop due to large percentage missing & not useful globally
data.drop(columns=['Postal_Code'], inplace=True)

# 3. Handling 'Returned':
# Option 1: Keep and label missing as "Not Known" (to allow grouping)
data['Returned'] = data['Returned'].fillna('Unknown')

# Optional: create binary flag for modeling later
data['Is_Returned'] = data['Returned'].apply(lambda x: 1 if x == 'Yes' else 0)

# 4. Confirm all missing handled
print("Missing Values After Handling:")
print(data.isnull().sum().sort_values(ascending=False).head())

Missing Values After Handling:
Order_ID               0
Sub-Category           0
Total_Order_Value      0
Profit_Margin          0
Shipping_Delay_Days    0
dtype: int64


Great! Let’s now calculate the Profit Margin, a key financial KPI that reveals how much profit is made relative to sales.

#### What is Profit Margin?

Profit Margin = (Proft /sales) * 100

It gives us a percentage value that shows the efficiency of sales in generating profit.

#### Add Profit_Margin Column

In [6]:
# Avoid division by zero
data = data[data['Sales'] != 0]

# Calculate Profit Margin (%)
data['Profit_Margin'] = (data['Profit'] / data['Sales']) * 100

# Round for readability
data['Profit_Margin'] = data['Profit_Margin'].round(2)

# Preview the new column
data[['Sales', 'Profit', 'Profit_Margin']].head()

,Sales,Profit,Profit_Margin
0,221.980,62.1544,28.00
1,3709.395,-288.7650,-7.78
2,5175.171,919.9710,17.78
3,2892.510,-96.5400,-3.34
4,2832.960,311.5200,11.00


#### Compute Total_Order_Value

In [7]:
# Total Order Value = Sales + Shipping_Cost
data['Total_Order_Value'] = data['Sales'] + data['Shipping_Cost']

# Optional: Round for clarity
data['Total_Order_Value'] = data['Total_Order_Value'].round(2)

# Show sample
data[['Sales', 'Shipping_Cost', 'Total_Order_Value']].head()

,Sales,Shipping_Cost,Total_Order_Value
0,221.980,40.77,262.75
1,3709.395,923.63,4633.02
2,5175.171,915.49,6090.66
3,2892.510,910.16,3802.67
4,2832.960,903.04,3736.00


In [8]:
cleaned_data = data
cleaned_data.to_csv("/Users/mynimbus/Library/Mobile Documents/com~apple~CloudDocs/GitHub Uploads/StoreSight/0_data/cleaned_data.csv")

In [9]:
# ✅ Final Data Cleaning Checks for cleaned_data (with updated column names)

# 1. General Info
print("=== DataFrame Info ===")
cleaned_data.info()

print("\n=== Summary Statistics ===")
print(cleaned_data.describe())

# 2. Missing Values Check
print("\n=== Missing Values Count ===")
print(cleaned_data.isnull().sum())

print("\n=== Missing Values Percentage ===")
print((cleaned_data.isnull().sum() / len(cleaned_data)) * 100)

# 3. Duplicates Check
print("\n=== Duplicate Rows Count ===")
print(cleaned_data.duplicated().sum())

# 4. Date Columns Check
print("\n=== Order and Ship Date Ranges ===")
print(f"Order Date: {cleaned_data['Order_Date'].min()} → {cleaned_data['Order_Date'].max()}")
print(f"Ship Date: {cleaned_data['Ship_Date'].min()} → {cleaned_data['Ship_Date'].max()}")

print("\n=== Date Column Data Types ===")
print(f"Order Date dtype: {cleaned_data['Order_Date'].dtype}")
print(f"Ship Date dtype: {cleaned_data['Ship_Date'].dtype}")

# 5. Value Validations
print("\n=== Rows with Sales <= 0 ===")
print(cleaned_data[cleaned_data['Sales'] <= 0])

print("\n=== Rows with Quantity <= 0 ===")
print(cleaned_data[cleaned_data['Quantity'] <= 0])

print("\n=== Sample Rows with Negative Profit (valid losses) ===")
print(cleaned_data[cleaned_data['Profit'] < 0].head())

# 6. Profit Margin Accuracy
print("\n=== Profit Margin Consistency Check ===")
cleaned_data['Calculated_Margin'] = cleaned_data['Profit'] / cleaned_data['Sales']
comparison_diff = (cleaned_data['Profit_Margin'] - cleaned_data['Calculated_Margin']).abs()
print(f"Max Difference in Profit Margin Calculation: {comparison_diff.max():.6f}")

# 7. Categorical Columns Review
print("\n=== Segment Distribution ===")
print(cleaned_data['Segment'].value_counts())

print("\n=== Ship Mode Distribution ===")
print(cleaned_data['Ship_Mode'].value_counts())

print("\n=== Order Priority Distribution ===")
print(cleaned_data['Order_Priority'].value_counts())

print("\n=== Returned Flag Distribution (Is_Returned) ===")
print(cleaned_data['Is_Returned'].value_counts())

# 8. Check Added Feature Columns
print("\n=== Sample of Added Columns ===")
print(cleaned_data[['Profit_Margin', 'Total_Order_Value', 'Is_Returned', 'Shipping_Delay_Days']].head())

# 9. Save Cleaned Data
#cleaned_data.to_csv('data/cleaned_global_superstore.csv', index=False)
print("\n✅ Cleaned data saved to 'data/cleaned_global_superstore.csv'")

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Order_ID             51290 non-null  object        
 1   Order_Date           51290 non-null  datetime64[ns]
 2   Ship_Date            51290 non-null  datetime64[ns]
 3   Ship_Mode            51290 non-null  object        
 4   Customer_ID          51290 non-null  object        
 5   Customer_Name        51290 non-null  object        
 6   Segment              51290 non-null  object        
 7   City                 51290 non-null  object        
 8   State                51290 non-null  object        
 9   Country              51290 non-null  object        
 10  Region               51290 non-null  object        
 11  Market               51290 non-null  object        
 12  Product_ID           51290 non-null  object        
 13  Category